In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
    sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation one our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [22]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: Lambda discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    nIterations = 0
    while True:
        print nIterations
        policy_stable = True
        V = policy_eval_fn(policy, env, discount_factor)
        
        for s in range(env.nS):
            current_best_a = np.argmax(policy[s])
            predicted_a_values = np.zeros(env.nA)
            for a in range(env.nA):
                for prob, next_state, reward, done in env.P[s][a]:
                    predicted_a_values[a] += prob * (reward + discount_factor * V[next_state])
            predicted_best_a = np.argmax(predicted_a_values)
            if current_best_a != predicted_best_a:
                policy_stable = False
            print s, predicted_a_values
            print 'before ', policy[s]
            print 'after ', np.eye(env.nA)[predicted_best_a]
            policy[s] = np.eye(env.nA)[predicted_best_a]
        if policy_stable:
            break
        
        nIterations += 1
    return policy, V

In [23]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



0
0 [ 0.  0.  0.  0.]
before  [ 0.25  0.25  0.25  0.25]
after  [ 1.  0.  0.  0.]
1 [-14.99993529 -20.99990698 -18.9999206   -1.        ]
before  [ 0.25  0.25  0.25  0.25]
after  [ 0.  0.  0.  1.]
2 [-20.99990698 -22.99989761 -20.99991379 -14.99993529]
before  [ 0.25  0.25  0.25  0.25]
after  [ 0.  0.  0.  1.]
3 [-22.99989761 -22.99989761 -20.99991477 -20.99990698]
before  [ 0.25  0.25  0.25  0.25]
after  [ 0.  0.  0.  1.]
4 [ -1.         -18.9999206  -20.99990698 -14.99993529]
before  [ 0.25  0.25  0.25  0.25]
after  [ 1.  0.  0.  0.]
5 [-14.99993529 -20.99991379 -20.99991379 -14.99993529]
before  [ 0.25  0.25  0.25  0.25]
after  [ 1.  0.  0.  0.]
6 [-20.99990698 -20.99991477 -18.99992725 -18.9999206 ]
before  [ 0.25  0.25  0.25  0.25]
after  [ 0.  0.  0.  1.]
7 [-22.99989761 -20.99991477 -14.99994569 -20.99991379]
before  [ 0.25  0.25  0.25  0.25]
after  [ 0.  0.  1.  0.]
8 [-14.99993529 -20.99991379 -22.99989761 -20.99990698]
before  [ 0.25  0.25  0.25  0.25]
after  [ 1.  0.  0.  0.]

In [12]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [13]:
print v

[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]
